In [ ]:
import pandas as pd
from RoutePlanner.Optimisation import TravelTime
from RoutePlanner import Plot
from RoutePlanner.TemporalCellGrid import TemporalCellGrid
import pickle

In [ ]:
OptInfo = {}

# Mesh Information
OptInfo['Mesh']  = {}
OptInfo['Mesh']['Longitude Bounds (Min,Max,Width)']      = [-130-0.0001,30-0.0001,5]
OptInfo['Mesh']['Latitude Bounds (Min,Max,Width)']       = [-80,-40,2.5]

OptInfo['Mesh']['Date Range (Min,Max,dT)']               = ['2013-01-1','2013-01-30',1.0]
OptInfo['Mesh']['Homogenous Params (Threshold,Min,Max)'] = [0.25,0.05,0.95] #[0.04,0.05,0.85] - Slow Vehicle, [0.12,0.05,0.85] - SDA
OptInfo['Mesh']['Current Data Path']                     = "../../Data/SOSE_surface_velocity_6yearMean_2005-2010.nc"
OptInfo['Mesh']['Ice Data Path']                         = "../../Data/bsose_i122_2013to2017_1day_SeaIceArea.nc"

# Route Information
OptInfo['Route'] = {} 
OptInfo['Route']['WayPoints']            = '../../resources/WayPoints_org.csv'
OptInfo['Route']['MaxIceExtent']         = 0.8
OptInfo['Route']['Zero Currents']        = True
OptInfo['Route']['VariableSpeed']        = False
OptInfo['Route']['Time Unit']            = 'days'

OptInfo['Route']['VehicleInfo']                = {}
OptInfo['Route']['VehicleInfo']['Speed']       = 26.5
OptInfo['Route']['VehicleInfo']['Unit']        = 'km/hr'
OptInfo['Route']['VehicleInfo']['Beam']        = 24.0
OptInfo['Route']['VehicleInfo']['HullType']    = 'slender'
OptInfo['Route']['VehicleInfo']['ForceLimit']  = 96634.5


In [ ]:
# Generating Cell Grid
temporalCellGrid = TemporalCellGrid(OptInfo)
cellGrid         = temporalCellGrid.range(OptInfo['Mesh']['Date Range (Min,Max,dT)'][0],OptInfo['Mesh']['Date Range (Min,Max,dT)'][1],j_grid=True)
cellGrid.cellBoxes[206].landLocked = True
cellGrid.iterativeSplit(3)
with open('cellGrid_split.p', 'wb') as f:
    pickle.dump(cellGrid,f)

with open('cellGrid_split.p', 'rb') as f:
    cellGrid = pickle.load(f)

# with open('cellGrid.p', 'rb') as f:
#     cellGrid = pickle.load(f)

In [ ]:
# # Difference - Land Mesh


In [ ]:
#cellGrid.plot()

map = Plot.BaseMap(logo=True,logoPos=[5,88])
map = Plot.MapMesh(cellGrid,map)
map = Plot.MapWaypoints(pd.read_csv(OptInfo['Route']['WayPoints']),map)
map = Plot.LayerControl(map,collapsed=True)
map

In [ ]:
TT = TravelTime(cellGrid,OptInfo)
RoutePaths = TT.Paths(source_waypoints=['LongPathStart'],end_waypoints=['LongPathEnd'],verbose=True,multiprocessing=False)

#RoutePaths = TT.Paths(source_waypoints=['MargueriteBay'],end_waypoints=['FilchnerTroughOverflow','NorthernWeddellSea','Brunt','Falklands','SouthGeorgia','AmundsenSea','LongPathEnd'],verbose=True,multiprocessing=False)
#RoutePaths = TT.Paths(source_waypoints=['MargueriteBay'],end_waypoints=['Falklands'],verbose=True,multiprocessing=False)

with open('Paths.p', 'wb') as f:
    pickle.dump(TT,f)

# with open('Paths.p', 'rb') as f:
#     TT = pickle.load(f)

In [ ]:
def MapMPaths(MPaths,map,color='blue',PathPoints=True):
    import folium
    import copy    
    Pths        = folium.FeatureGroup(name='Paths')
    Pths_points = folium.FeatureGroup(name='Path Points')
    for path in copy.deepcopy(RoutePaths):
        points = MPaths
        # points[:,0] = points[:,0]
        points = points[:,::-1]
        folium.PolyLine(points,color=color, weight=2.0, opacity=1).add_to(Pths)
        for idx in range(len(points)):
            loc = [points[idx,0],points[idx,1]]
            folium.Marker(
                location=loc,
                icon=folium.plugins.BeautifyIcon(icon='circle',
                                            border_color='transparent',
                                            background_color='transparent',
                                            border_width=1,
                                            text_color=color,
                                            inner_icon_style='margin:0px;font-size:0.8em')
            ).add_to(Pths_points)
    Pths.add_to(map)
    if PathPoints:
        Pths_points.add_to(map)
    return map

def JavaPaths(file):
    import json
    import numpy as np
    with open(file, 'r') as f:
        JavaGeo = json.load(f)['allpaths']['January']

    PathPts = []
    for path in JavaGeo:
        if any(path['from'] in s for s in TT.source_waypoints) and any(path['to'] in s for s in TT.end_waypoints):
            print(path['from'],path['to'])
            pts=[]
            for jj in path['path']:
                try:
                    pts.append([jj['lon'],-jj['lat']])
                except:
                    continue
            pts = np.array(pts)
            PathPts.append(pts)
    return PathPts


map = Plot.BaseMap(logo=True,logoPos=[5,88])
#map = Plot.MapMesh(cellGrid,map,threshold=OptInfo['Route']['MaxIceExtent'])
map = Plot.MapWaypoints(pd.read_csv(OptInfo['Route']['WayPoints']),map)

for mPath in JavaPaths('unsmoothedpathDetails2013.json'):
    map = MapMPaths(mPath,map,color='blue',PathPoints=True)

map = Plot.MapPaths(RoutePaths,map,PathPoints=False)
map = Plot.LayerControl(map,collapsed=True)
map

In [ ]:
import numpy as np
from shapely.geometry import Polygon

class _Euclidean_distance():
    """
    Replicating original route planner Euclidean distance 
    Inputs:
      origin      - tuple of floats e.g. (Long_orig,Lat_orig)
      destination - tuple of floats e.g. (Long_dest,Lat_dest)
      Optional: forward - Boolean True or False
    Output:
      Value - If 'forward' is True then returns Distance between 
              two points in 'km'. If 'False' then return the 
              Lat/Long position of a point.

    """

    def __init__(self,scaleLongitude=None):

        self.m_per_latitude  = 111.386*1000.

        if type(scaleLongitude) != type(None):
            self.m_per_longitude = 111.321*1000#*np.cos(scaleLongitude)#*(np.pi/180))
        else:
            self.m_per_longitude = (111.321*1000.)

    def value(self,origin,dest_dist,forward=True):
        lon1,lat1 = origin
        if forward:
            lon2,lat2 = dest_dist
            lon2 = lon2+360
            lon1 = lon1+360
            val = np.sqrt(((lat2-lat1)*self.m_per_latitude)**2 + ((lon2-lon1)*self.m_per_longitude)**2)
        else:
            dist_x,dist_y = dest_dist        
            val = [lon1+(dist_x/self.m_per_longitude),lat1+(dist_y/self.m_per_latitude)]
        return val

class NewtonianCurve:
    def __init__(self,Mesh,DijkstraInfo,OptInfo,unit_shipspeed='km/hr',unit_time='days',debugging=0,maxiter=1000,pathIter=5,optimizer_tol=1e-3,minimumDiff=1e-3,zerocurrents=True):
        '''
        
        
            BUG:
                - Currently the speed is fixed. Move the construction of the cellBox speed to a function of the cellBox
        
        '''

        # Passing the Mesh information
        self.Mesh = Mesh

        # Passing the Dijkstra Graph
        self.DijkstraInfo = copy.copy(DijkstraInfo)

        # Passing the optional Information
        self.OptInfo = OptInfo

        
        # Inside the code the base units are m/s. Changing the units of the inputs to match
        self.unit_shipspeed = 'km/hr'#unit_shipspeed
        self.unit_time      = unit_time
        self.s              = self._unit_speed(26.5)
        
        # Information for distance metrics
        self.R              = 6371.*1000
        self.fdist          = _Euclidean_distance()

        # Optimisation Information
        self.maxiter       = maxiter
        self.pathIter      = pathIter
        self.optimizer_tol = optimizer_tol
        self.minimumDiff   = minimumDiff
        self._epsilon      = 1e-6


        # For Debugging purposes 
        self.debugging     = debugging

        self.id = 0

        # zeroing currents if flagged
        if zerocurrents:
            self.zc = 0.0
        else:
            self.zc = 1.0

    def _unit_speed(self,Val):
        if self.unit_shipspeed == 'km/hr':
            Val = Val*(1000/(60*60))
        if self.unit_shipspeed == 'knots':
            Val = (Val*0.51)
        return Val

    def _unit_time(self,Val):
        if self.unit_time == 'days':
            Val = Val/(60*60*24)
        elif self.unit_time == 'hr':
            Val = Val/(60*60)
        elif self.unit_time == 'min':
            Val = Val/(60)
        elif self.unit_time == 's':
            Val = Val
        return Val

    def _long_case(self):
            def NewtonOptimisationLong(f,y0,x,a,Y,u1,v1,u2,v2,s,R,λ_s,φ_r):
                    tryNum=1
                    iter=0
                    if self.maxiter > 0:
                        while iter < self.maxiter:
                            # # -- Maria Original
                            # if (iter>100) and (tryNum==1):
                            #     y0 = (Y*x)/(x+a)
                            #     tryNum+=1
                            # if (iter>200) and (tryNum>=2) and (tryNum <10):
                            #     tryNum+=1
                            #     iter-=100
                            #     if (Y<0):
                            #         if v2>v1:
                            #             y0 = (tryNum-2)*Y
                            #         else:
                            #             y0 = (tryNum-3)*-Y
                            #     else:
                            #         if v2<v1:
                            #             y0 = (tryNum-2)*Y
                            #         else:
                            #             y0 = (tryNum-3)*-Y        

                            F,dF,X1,X2  = f(y0,x,a,Y,u1,v1,u2,v2,s,R,λ_s,φ_r)
                            
                            if (F==0) or (dF==0):
                                dY = 0
                            else:
                                dY = (F/dF)
                            
                            imprving =  (abs(dY)>self._epsilon)
                            if not imprving:
                                break
                            y0  -= dY
                            iter+=1
                    return y0

            def _F(y,x,a,Y,u1,v1,u2,v2,s,R,λ_s,φ_r):
                θ  = (y/R + λ_s*(np.pi/180))
                zl = x*np.cos(θ)
                ψ  = (-(Y-y)/R + φ_r*(np.pi/180))
                zr = a*np.cos(ψ)

                C1  = s**2 - u1**2 - v1**2
                C2  = s**2 - u2**2 - v2**2
                D1  = zl*u1 + y*v1
                D2  = zr*u2 + (Y-y)*v2
                X1  = np.sqrt(D1**2 + C1*(zl**2 + y**2))
                X2  = np.sqrt(D2**2 + C2*(zr**2 + (Y-y)**2))

                dzr = -zr*np.sin(ψ)/R
                dzl = -zl*np.sin(θ)/R

                dD1 = dzl*u1 + v1
                dD2 = dzr*u2 - v2
                dX1 = (D1*v1 + C1*y + dzl*(D1*u1 + C1*zl))/X1
                dX2 = (-v2*D2 - C2*(Y-y) + dzr*(D2*u2 + C2*zr))/X2     

                zr_term = (zr - (X2 - D2)*u2/C2)
                zl_term = (zl - (X1 - D1)*u1/C1)

                F = (X1+X2)*y - v1*(X1-D1)*X2/C1 - (Y - v2*(X2-D2)/C2)*X1 + dzr*zr_term*X1 + dzl*zl_term*X2

                dF = (X1+X2) + y*(dX1 + dX2) - (v1/C1)*(dX2*(X1-D1) + X2*(dX1-dD1))\
                    - Y*dX1 + (v2/C2)*(dX1*(X2-D2) + X1*(dX2-dD2))\
                    - (zr/(R**2))*zr_term*X1\
                    - (zl/(R**2))*zl_term*X2\
                    + dzr*(dzr-u2*(dX2-dD2))/C2*X1\
                    + dzl*(dzl-u1*(dX1-dD1))/C1*X2\
                    + dzr*zr_term*dX1 + dzl*zl_term*dX2

                return F,dF,X1,X2

            Cp = tuple(self.triplet[['cX','cY']].iloc[1])
            if self.triplet.iloc[1].case == 2:   
                Sp   = tuple(self.triplet.iloc[0][['cX','cY']])
                Np   = tuple(self.triplet.iloc[2][['cX','cY']])
                Box1 = self.Mesh.cellBoxes[self.triplet.iloc[1]['cellStart'].name]
                Box2 = self.Mesh.cellBoxes[self.triplet.iloc[1]['cellEnd'].name]
                sgn  = 1
            else:
                Sp   = tuple(self.triplet.iloc[2][['cX','cY']])
                Np   = tuple(self.triplet.iloc[0][['cX','cY']])
                Box1 = self.Mesh.cellBoxes[self.triplet.iloc[1]['cellEnd'].name]
                Box2 = self.Mesh.cellBoxes[self.triplet.iloc[1]['cellStart'].name]
                sgn  = -1

            λ_s  = Sp[1]
            φ_r  = Np[1]

            x           = self.fdist.value(Sp,(Cp[0],Sp[1]))
            a           = self.fdist.value(Np,(Cp[0],Np[1]))

            Y           = np.sign(Np[1]-Sp[1])*self.fdist.value((Sp[0],Sp[1]),(Sp[0],Np[1]))
            y0          = Y/2
            y0          = np.sign(Cp[1]-Sp[1])*self.fdist.value((Cp[0],Sp[1]),(Cp[0],Cp[1]))

            u1          = sgn*self.zc*Box1.getuC(); v1 = self.zc*Box1.getvC()
            u2          = sgn*self.zc*Box2.getuC(); v2 = self.zc*Box2.getvC()
            y           = NewtonOptimisationLong(_F,y0,x,a,Y,u1,v1,u2,v2,self.s,self.R,λ_s,φ_r)

            # Updating the crossing points
            CrossP = self.fdist.value((Cp[0],Sp[1]),(0.0,y),forward=False)
            self.triplet['cX'].iloc[1] = CrossP[0]
            self.triplet['cY'].iloc[1] = CrossP[1]

            # if abs((self.triplet['cX'].iloc[1]-self.triplet['cX'].iloc[0])) < 1e-4 and abs((self.triplet['cY'].iloc[1]-self.triplet['cY'].iloc[0])) >1e-4:
            #     self.triplet['cX'].iloc[0] = self.triplet['cX'].iloc[1]
            #     self.triplet = self.triplet.drop(self.triplet.iloc[1].name).sort_index()
            # if abs((self.triplet['cX'].iloc[1]-self.triplet['cX'].iloc[2])) < 1e-4 and abs((self.triplet['cY'].iloc[1]-self.triplet['cY'].iloc[2])) >1e-4:
            #     self.triplet['cX'].iloc[2] = self.triplet['cX'].iloc[1]
            #     self.triplet = self.triplet.drop(self.triplet.iloc[1].name).sort_index()

    def _lat_case(self):
        def NewtonOptimisationLat(f,y0,x,a,Y,u1,v1,u2,v2,s,R,λ,θ,ψ):
                tryNum=1
                iter=0
                while iter < 1000:
                    # # -- Maria Original
                    # if (iter>100) and (tryNum==1):
                    #     y0 = (Y*x)/(x+a)
                    #     tryNum+=1
                    # if (iter>200) and (tryNum>=2) and (tryNum <10):
                    #     tryNum+=1
                    #     iter-=100
                    #     if (Y<0):
                    #         if v2>v1:
                    #             y0 = (tryNum-2)*Y
                    #         else:
                    #             y0 = (tryNum-3)*-Y
                    #     else:
                    #         if v2<v1:
                    #             y0 = (tryNum-2)*Y
                    #         else:
                    #             y0 = (tryNum-3)*-Y        

                    # --- Updating ---
                    F,dF,X1,X2  = f(y0,x,a,Y,u1,v1,u2,v2,s,R,λ,θ,ψ)
                    
                    if (F==0) or (dF==0):
                        dY = 0
                    else:
                        dY = (F/dF)
                    
                    imprving = (abs(dY)>self._epsilon)
                    

                    # print('Iter={}:F={},dF={},dY={},imprving={},y0={},Y={}\n'.format(iter,F,dF,dY,imprving))

                    if imprving:
                        y0 -= dY
                        iter +=1
                    else:
                        break

                return y0

        def _F(y,x,a,Y,u1,v1,u2,v2,s,R,λ,θ,ψ):
            λ   = λ*(np.pi/180)
            ψ   = ψ*(np.pi/180)
            θ   = θ*(np.pi/180)
            r1  = np.cos(λ)/np.cos(θ)
            r2  = np.cos(ψ)/np.cos(θ)

            d1  = np.sqrt(x**2 + (r1*y)**2)
            d2  = np.sqrt(a**2 + (r2*(Y-y))**2)
            C1  = s**2 - u1**2 - v1**2
            C2  = s**2 - u2**2 - v2**2
            D1  = x*u1 + r1*v1*y
            D2  = a*u2 + r2*v2*(Y-y)
            X1  = np.sqrt(D1**2 + C1*(d1**2))
            X2  = np.sqrt(D2**2 + C2*(d2**2)) 

            dX1 = (r1*(D1*v1 + r1*C1*y))/X1
            dX2 = (-r2*(D2*v2 + r2*C2*(Y-y)))/X2

            F = ((r2**2)*X1+(r1**2)*X2)*y - r1*v1*(X1-D1)*X2/C1 - r2*(r2*Y-v2*(X2-D2)/C2)*X1

            dF = ((r2**2)*X1 + (r1**2)*X2) + y*((r2**2)*dX1 + (r1**2)*dX2)\
                - r1*v1*((X1-D1)*dX2 + (dX1-r1*v1)*X2)/C1\
                - (r2**2)*Y*dX1\
                + r2*v2*((X2-D2)*dX1 + (dX2+r2*v2)*X1)/C2

            return F,dF,X1,X2


        Cx,Cy = tuple(self.triplet.iloc[1][['cX','cY']])
        if self.triplet.iloc[1].case == -4:   
            Sx,Sy = tuple(self.triplet.iloc[0][['cX','cY']])
            Nx,Ny = tuple(self.triplet.iloc[2][['cX','cY']])
            Cl1   = self.Mesh.cellBoxes[self.triplet.iloc[1]['cellStart'].name]
            Cl2   = self.Mesh.cellBoxes[self.triplet.iloc[1]['cellEnd'].name]
            sgn   = 1
        else:
            Sx,Sy = tuple(self.triplet.iloc[0][['cX','cY']])
            Nx,Ny = tuple(self.triplet.iloc[2][['cX','cY']])
            Cl1   = self.Mesh.cellBoxes[self.triplet.iloc[1]['cellStart'].name]
            Cl2   = self.Mesh.cellBoxes[self.triplet.iloc[1]['cellEnd'].name] 
            sgn   = -1

        λ=Sy
        θ=Cy   
        ψ=Ny  

        x     = self.fdist.value((Sx,Sy),(Sx,Cy))
        a     = self.fdist.value((Nx,Ny),(Nx,Cy))
        Y     = np.sign(Nx-Sx)*self.fdist.value((Sx,Sy),(Nx,Sy))
        Su    = sgn*self.zc*Cl1.getvC(); Sv = self.zc*Cl1.getuC()
        Nu    = sgn*self.zc*Cl2.getvC(); Nv = self.zc*Cl2.getuC()
        
        y0    = Y/2
        y0    = np.sign(Cx-Sx)*self.fdist.value((Sx,Cy),(Cx,Cy))

        y     = NewtonOptimisationLat(_F,y0,x,a,Y,Su,Sv,Nu,Nv,self.s,self.R,λ,θ,ψ)

        CrossP = self.fdist.value((Sx,Cy),(y,0.0),forward=False)
        self.triplet['cX'].iloc[1] = CrossP[0]
        self.triplet['cY'].iloc[1] = CrossP[1]



    def _corner_case(self):
        '''
        '''
        # Separting out the Long/Lat of each of the points
        Xs,Ys = tuple(self.triplet.iloc[0][['cX','cY']])
        Xc,Yc = tuple(self.triplet.iloc[1][['cX','cY']])
        Xe,Ye = tuple(self.triplet.iloc[2][['cX','cY']])

        # === 1. Assess the cells that are shared commonly in the corner case ====
        sourceNeighbourIndices = self.triplet.iloc[1]['cellStart']
        endNeighbourIndices    = self.triplet.iloc[1]['cellEnd']
    
        commonIndices = list(set(sourceNeighbourIndices['neighbourIndex']).intersection(endNeighbourIndices['neighbourIndex']))
        CornerCells   = self.DijkstraInfo.loc[commonIndices]
        Y_line = ((Ye-Ys)/(Xe-Xs))*(Xc-Xs) + Ys

        # if np.sign(self.triplet['case'].iloc[1]) == -1:

        if Yc >= Y_line:
            newCell = CornerCells.loc[CornerCells['cY'].idxmin()]
            if newCell.cY > Yc:
                return
        elif Yc < Y_line:
            newCell = CornerCells.loc[CornerCells['cY'].idxmax()]
            if newCell.cY < Yc:
                return

        # === 3. Return the path crossing points and cell indices
        try:
            firstCrossingPoint  = np.array(sourceNeighbourIndices['neighbourCrossingPoints'])[np.where(np.array(sourceNeighbourIndices['neighbourIndex'])==newCell.name)[0][0],:]
            secondCrossingPoint = np.array(newCell['neighbourCrossingPoints'])[np.where(np.array(newCell['neighbourIndex'])==endNeighbourIndices.name)[0][0],:]
        except:
            self.triplet = copy.deepcopy(self.org_triplet)
            return

        # Adding in the new crossing Point
        newP = pd.Series(name=self.triplet.iloc[1].name+1)
        newP['cX']        = secondCrossingPoint[0]
        newP['cY']        = secondCrossingPoint[1]
        newP['cellStart'] = newCell
        newP['cellEnd']   = copy.deepcopy(self.triplet['cellEnd'].iloc[1])
        newP['case']      = newP['cellStart']['case'][np.where(np.array(newP['cellStart']['neighbourIndex'])==newP['cellEnd'].name)[0][0]]


        # Updating the origional crossing point
        self.triplet['cX'].iloc[1]      = firstCrossingPoint[0]
        self.triplet['cY'].iloc[1]      = firstCrossingPoint[1]
        self.triplet['cellEnd'].iloc[1] = newCell 
        self.triplet['case'].iloc[1]    = self.triplet['cellStart'].iloc[1]['case'][np.where(np.array(self.triplet['cellStart'].iloc[1]['neighbourIndex'])==newCell.name)[0][0]]


        # Adding the new crossing point to the triplet
        self.CrossingDF = self.CrossingDF.append(newP,sort=True).sort_index().reset_index(drop=True)
        self.CrossingDF.index = np.arange(int(self.CrossingDF.index.min()),int(self.CrossingDF.index.max()*1e3 + 1e3),int(1e3))


    def _mergePoint(self):
        '''
            Function to merge point if on the corner 
        '''
        #1. Find indices of points that are identical
        Dist = np.sqrt((self.CrossingDF['cX'][1:].to_numpy() - self.CrossingDF['cX'][:-1].to_numpy())**2 + (self.CrossingDF['cY'][1:].to_numpy() - self.CrossingDF['cY'][:-1].to_numpy())**2)
        idx = np.where(Dist<=1e-4)[0]

        
        if len(idx) != 0:

            #2. Remove the row and update thr case information and crossing point
            for id in idx:
                # if (id==0) or (id==len(Dist)-2):
                #     return


                neighbourIndex=np.where(np.array(self.CrossingDF.iloc[id]['cellStart']['neighbourIndex'])==self.CrossingDF.iloc[id+1]['cellEnd'].name)[0][0]
                
                case = self.CrossingDF['cellStart'].iloc[id]['case'][neighbourIndex]
                crossingPoint = self.CrossingDF['cellStart'].iloc[id]['neighbourCrossingPoints'][neighbourIndex]


                print('Merge Point Case={},id={},lenDist={}'.format(case,id,len(Dist)))

                #if abs(case)==1 or abs(case)==3:
                self.CrossingDF['cX'].iloc[id]      = crossingPoint[0]
                self.CrossingDF['cY'].iloc[id]      = crossingPoint[1]
                self.CrossingDF['cellEnd'].iloc[id] = copy.deepcopy(self.CrossingDF.iloc[id+1]['cellEnd'])
                self.CrossingDF['case'].iloc[id] = copy.deepcopy(case)
                self.CrossingDF = self.CrossingDF.drop(self.CrossingDF.iloc[id+1].name).sort_index().reset_index(drop=True)
                self.CrossingDF.index = np.arange(int(self.CrossingDF.index.min()),int(self.CrossingDF.index.max()*1e3 + 1e3),int(1e3))


    def _horseshoe(self):
        '''
            1. Determine the case of the problem investigating and outline the start and end indices

            2. List the neighbours for the start and end indices that satisfy that case. 
            If they share a common neighbour then a horseshoe cannot be generated & we have an additional case.
            
            3. From those neighbours, determine their subsequent neighbours determining if any of these lists share a common entry

            4. Determine the pair that shares the common entry

            5. Re-inspect the graph adding the crossing points between this pair, adding back in the horseshoe
        '''

        # Defining the case information
        Cp             = tuple(self.triplet[['cX','cY']].iloc[1])
        Sp             = tuple(self.triplet.iloc[0][['cX','cY']])
        Np             = tuple(self.triplet.iloc[2][['cX','cY']])
        cellStart      = self.Mesh.cellBoxes[self.triplet.iloc[1]['cellStart'].name]
        cellStartGraph = self.triplet.iloc[1]['cellStart']
        cellEnd        = self.Mesh.cellBoxes[self.triplet.iloc[1]['cellEnd'].name]
        cellEndGraph   = self.triplet.iloc[1]['cellEnd']
        case           = self.triplet['case'].iloc[1]

        # Returning if corner horseshoe case type
        if abs(case)==1 or abs(case)==3 or abs(case)==0: 
            return
        elif abs(case) == 2:

            # Defining the global min and max
            gmin = np.min([cellStart.cy-cellStart.dcy,cellEnd.cy-cellEnd.dcy])
            gmax = np.max([cellStart.cy+cellStart.dcy,cellEnd.cy+cellEnd.dcy])
            vmin = np.max([cellStart.cy-cellStart.dcy,cellEnd.cy-cellEnd.dcy])
            vmax = np.min([cellStart.cy+cellStart.dcy,cellEnd.cy+cellEnd.dcy])

            # Point crossingpoint on boundary between the two origional cells
            if (Cp[1] > vmin) and (Cp[1] < vmax):
                return

            # Defining the min and max of the start and end cells
            smin = cellStart.cy-cellStart.dcy   
            smax = cellStart.cy+cellStart.dcy
            emin = cellEnd.cy-cellEnd.dcy
            emax = cellEnd.cy+cellEnd.dcy

            # If Start and end cells share a edge for the horesshoe 
            if (Cp[1]<=smin) and (smin==emin):
                hrshCaseStart = 4
                hrshCaseEnd   = 4
            if (Cp[1]>=smax) and (smax==emax):
                hrshCaseStart = -4
                hrshCaseEnd   = -4

            # --- Cases where StartCell is Larger than end Cell ---
            if (Cp[1]>=emax) and (smax>emax):
                hrshCaseStart = case
                hrshCaseEnd   = 4                
            if (Cp[1]<=emin) and (smin<emin):
                hrshCaseStart = case
                hrshCaseEnd   = -4                   

            # --- Cases where StartCell is smaller than end Cell ---
            if (Cp[1]>=smax) and (smax<emax):
                hrshCaseStart = -4
                hrshCaseEnd   = case
            if (Cp[1]<=smin) and (emin<smin):
                hrshCaseStart = 4
                hrshCaseEnd   = case                    

        elif abs(case) == 4:

            # Defining the global min and max
            gmin = np.min([cellStart.cx-cellStart.dcx,cellEnd.cx-cellEnd.dcx])
            gmax = np.max([cellStart.cx+cellStart.dcx,cellEnd.cx+cellEnd.dcx])
            vmin = np.max([cellStart.cx-cellStart.dcx,cellEnd.cx-cellEnd.dcx])
            vmax = np.min([cellStart.cx+cellStart.dcx,cellEnd.cx+cellEnd.dcx])

            # Point crossingpoint on boundary between the two origional cells
            if (Cp[0] > vmin) and (Cp[0] < vmax):
                return

            # Defining the min and max of the start and end cells
            smin = cellStart.cx-cellStart.dcx   
            smax = cellStart.cx+cellStart.dcx
            emin = cellEnd.cx-cellEnd.dcx
            emax = cellEnd.cx+cellEnd.dcx


            # If Start and end cells share a edge for the horesshoe 
            if (Cp[0]<smin) and (smin==emin):
                hrshCaseStart = -2
                hrshCaseEnd   = -2
            if (Cp[0]>smax) and (smax==emax):
                hrshCaseStart = 2
                hrshCaseEnd   = 2

            # --- Cases where StartCell is Larger than end Cell ---
            if (Cp[0]>emax) and (smax>emax):
                hrshCaseStart = case
                hrshCaseEnd   = -2                
            if (Cp[1]<emin) and (smin<emin):
                hrshCaseStart = case
                hrshCaseEnd   = 2                   

            # --- Cases where StartCell is smaller than end Cell ---
            if (Cp[0]>smax) and (smax<emax):
                hrshCaseStart = 2
                hrshCaseEnd   = case
            if (Cp[0]<smin) and (emin<smin):
                hrshCaseStart = -2
                hrshCaseEnd   = case   


        # Determining the neighbours of the start and end cells that are the horseshoe case
        startGraphNeighbours = [cellStartGraph['neighbourIndex'][ii] for ii in list(np.where(np.array(cellStartGraph['case'])==hrshCaseStart)[0])]
        endGraphNeighbours   = [cellEndGraph['neighbourIndex'][ii] for ii in list(np.where(np.array(cellEndGraph['case'])==hrshCaseEnd)[0])]

        if (len(startGraphNeighbours)==0) or (len(endGraphNeighbours)==0):
            if abs(case) == 2:
                self.triplet['cY'].iloc[1] = np.clip(self.triplet.iloc[1]['cY'],vmin+1e-4,vmax-1e-4)
            if abs(case) == 4:
                self.triplet['cX'].iloc[1] = np.clip(self.triplet.iloc[1]['cX'],vmin+1e-4,vmax-1e-4)        
            return
        
        if abs(hrshCaseStart) == abs(hrshCaseEnd):
            for sGN in startGraphNeighbours:
                for eGN in endGraphNeighbours:
                    if (np.array(self.DijkstraInfo.loc[sGN,'neighbourIndex'])==eGN).any() and (np.array(self.DijkstraInfo.loc[eGN,'neighbourIndex'])==sGN).any():
                        sGNGraph = self.DijkstraInfo.loc[sGN]
                        eGNGraph = self.DijkstraInfo.loc[eGN]

                        Crp1 = np.array(cellStartGraph['neighbourCrossingPoints'])[np.where(np.array(cellStartGraph['neighbourIndex']) == sGN)[0][0],:]
                        Crp2 = np.array(sGNGraph['neighbourCrossingPoints'])[np.where(np.array(sGNGraph['neighbourIndex']) == eGN)[0][0],:]
                        Crp3 = np.array(eGNGraph['neighbourCrossingPoints'])[np.where(np.array(eGNGraph['neighbourIndex']) == cellEndGraph.name)[0][0],:]
                        


                        # Updating the origional crossing point
                        self.triplet['cX'].iloc[1]      = Crp1[0]
                        self.triplet['cY'].iloc[1]      = Crp1[1]
                        self.triplet['cellEnd'].iloc[1] = copy.deepcopy(sGNGraph)
                        self.triplet['case'].iloc[1]    = self.triplet['cellStart'].iloc[1]['case'][np.where(np.array(self.triplet['cellStart'].iloc[1]['neighbourIndex'])==sGNGraph.name)[0][0]]

                        # Crossing Point 2
                        Pcrp2 = pd.Series(name=self.triplet.iloc[1].name+1)
                        Pcrp2['cX']        = Crp2[0]
                        Pcrp2['cY']        = Crp2[1]
                        Pcrp2['cellStart'] = copy.deepcopy(sGNGraph)
                        Pcrp2['cellEnd']   = copy.deepcopy(eGNGraph)
                        Pcrp2['case']      = Pcrp2['cellStart']['case'][np.where(np.array(Pcrp2['cellStart']['neighbourIndex'])==Pcrp2['cellEnd'].name)[0][0]]

                        Pcrp3 = pd.Series(name=self.triplet.iloc[1].name+2)
                        Pcrp3['cX']        = Crp3[0]
                        Pcrp3['cY']        = Crp3[1]
                        Pcrp3['cellStart'] = copy.deepcopy(eGNGraph)
                        Pcrp3['cellEnd']   = copy.deepcopy(cellEndGraph)
                        Pcrp3['case']      = Pcrp3['cellStart']['case'][np.where(np.array(Pcrp3['cellStart']['neighbourIndex'])==Pcrp3['cellEnd'].name)[0][0]]
                        
                        self.CrossingDF = self.CrossingDF.append([Pcrp2,Pcrp3],sort=True).sort_index().reset_index(drop=True)
                        self.CrossingDF.index = np.arange(int(self.CrossingDF.index.min()),int(self.CrossingDF.index.max()*1e3 + 1e3),int(1e3))

                        # self.id=+1
        else:
            for sGN in startGraphNeighbours:
                for eGN in endGraphNeighbours:
                    if (np.array(sGN==eGN).any()):
                        print('SUCESS',sGN,eGN)
                        NeighGraph = self.DijkstraInfo.loc[sGN]               
                        Crp1 = np.array(cellStartGraph['neighbourCrossingPoints'])[np.where(np.array(cellStartGraph['neighbourIndex']) == sGN)[0][0],:]
                        Crp2 = np.array(NeighGraph['neighbourCrossingPoints'])[np.where(np.array(NeighGraph['neighbourIndex']) == cellEndGraph.name)[0][0],:]


                        # Updating the origional crossing point
                        self.triplet['cX'].iloc[1]      = Crp1[0]
                        self.triplet['cY'].iloc[1]      = Crp1[1]
                        self.triplet['cellEnd'].iloc[1] = copy.deepcopy(NeighGraph)
                        self.triplet['case'].iloc[1]    = self.triplet['cellStart'].iloc[1]['case'][np.where(np.array(self.triplet['cellStart'].iloc[1]['neighbourIndex'])==NeighGraph.name)[0][0]]

                        Pcrp2 = pd.Series(name=self.triplet.iloc[1].name+2)
                        Pcrp2['cX']        = Crp2[0]
                        Pcrp2['cY']        = Crp2[1]
                        Pcrp2['cellStart'] = copy.deepcopy(NeighGraph)
                        Pcrp2['cellEnd']   = copy.deepcopy(cellEndGraph)
                        Pcrp2['case']      = Pcrp2['cellStart']['case'][np.where(np.array(Pcrp2['cellStart']['neighbourIndex'])==Pcrp2['cellEnd'].name)[0][0]]
                        
                        self.CrossingDF = self.CrossingDF.append([Pcrp2],sort=True).sort_index().reset_index(drop=True)
                        self.CrossingDF.index = np.arange(int(self.CrossingDF.index.min()),int(self.CrossingDF.index.max()*1e3 + 1e3),int(1e3))

                        # self.id=+1

    def _reverseCase(self):

        # Removing Reverse Edge Type 1
        startIndex = np.array([row['cellStart'].name for idx,row in self.CrossingDF.iterrows()][1:-1])
        endIndex   = np.array([row['cellEnd'].name for idx,row in self.CrossingDF.iterrows()][1:-1] )
        boolReverseEdge  = np.logical_and((startIndex[:-1] == endIndex[1:]),(startIndex[1:] == endIndex[:-1]))
        if boolReverseEdge.any():
            indxReverseEdge = np.where(boolReverseEdge)[0]+1
            for id in indxReverseEdge:
                self.CrossingDF = self.CrossingDF.drop(self.CrossingDF.iloc[id].name).sort_index().reset_index(drop=True)


        # Removing Reverse Edge Type 2
        startIndex = np.array([row['cellStart'].name for idx,row in self.CrossingDF.iterrows()][1:-1])
        endIndex   = np.array([row['cellEnd'].name for idx,row in self.CrossingDF.iterrows()][1:-1] )
        boolReverseEdge  = (endIndex[:-1] == endIndex[1:])
        if boolReverseEdge.any():
            indxReverseEdge = np.where(boolReverseEdge)[0]+2
            for id in indxReverseEdge:
                self.CrossingDF = self.CrossingDF.drop(nc.CrossingDF.iloc[id].name).sort_index().reset_index(drop=True)


        self.CrossingDF.index = np.arange(0,int(len(self.CrossingDF)*1e3),int(1e3))


    def _updateCrossingPoint(self):
        '''
            COMPLETE:
                - Unsmoothed _long_case_ Path, Unsplit & No Currents - 
        '''


        self.org_triplet = copy.deepcopy(self.triplet) 


        # ------ Case Deginitions & Dealing
        if self.debugging>0:
            print('===========================================================')
        if abs(self.triplet.iloc[1].case)==2:
            self._long_case()
            self.id=0
        elif abs(self.triplet.iloc[1].case)==4:
            self._lat_case()
            self.id=0
        elif (abs(self.triplet.iloc[1].case)==1) or (abs(self.triplet.iloc[1].case)==3):
            self._corner_case()
            #self.id=-1

        if len(self.triplet) < 3:
            return

        # # -- Horseshoe Case Detection -- 
        self._horseshoe()

        # # -- Removing reseverse cases
        self._reverseCase()




In [ ]:
import copy
from matplotlib.patches import Polygon as MatplotPolygon


numberIterations = 300
with open('Paths.p', 'rb') as f:
    TT = pickle.load(f)

Paths = []
for ii in range(len(TT.paths)):
    Path = copy.copy(TT.paths[ii])
    print(Path['to'])

    nc = NewtonianCurve(TT.Mesh,TT.DijkstraInfo[Path['from']],TT.OptInfo,maxiter=1,zerocurrents=True)
    nc.pathIter = numberIterations

    # -- Generating a dataframe of the case information -- 
    Points      = np.concatenate([Path['Path']['Points'][0,:][None,:],Path['Path']['Points'][1:-1:2],Path['Path']['Points'][-1,:][None,:]])
    cellIndices = np.concatenate([Path['Path']['CellIndices'][0][None],Path['Path']['CellIndices'],Path['Path']['CellIndices'][-1][None]])
    cellDijk    = [nc.DijkstraInfo.loc[ii] for ii in cellIndices]
    nc.CrossingDF  = pd.DataFrame({'cX':Points[:,0],'cY':Points[:,1],'cellStart':cellDijk[:-1],'cellEnd':cellDijk[1:]})

    # -- Determining the cases from the cell information. If within cell then case 0 -- 
    Cases = []
    for idx,row in nc.CrossingDF.iterrows():
        try:
            Cases.append(row.cellStart['case'][np.where(np.array(row.cellStart['neighbourIndex'])==row.cellEnd.name)[0][0]])
        except:
            Cases.append(0)
    nc.CrossingDF['case'] = Cases
    nc.CrossingDF.index = np.arange(int(nc.CrossingDF.index.min()),int(nc.CrossingDF.index.max()*1e3 + 1e3),int(1e3))

    nc.orgDF = copy.deepcopy(nc.CrossingDF)
    iter=0
    try:
        while iter < nc.pathIter:
            print('===== Iter {} ====='.format(iter))

            nc.previousDF = copy.deepcopy(nc.CrossingDF)
            id = 0
            while id <= (len(nc.CrossingDF) - 3):
                nc.triplet = nc.CrossingDF.iloc[id:id+3]


                nc._updateCrossingPoint()
                id+=1+nc.id

            nc._mergePoint()

            iter+=1

            # Stop optimisation if the points are within some minimum difference
            if iter>15:
                if len(nc.previousDF) == len(nc.CrossingDF):
                    Dist = np.mean(np.sqrt((nc.previousDF['cX'] - nc.CrossingDF['cX'])**2 + (nc.previousDF['cY'] - nc.CrossingDF['cY'])**2))
                    if Dist < 1e-4:
                        break
    except:
        print('')



    Paths.append(copy.deepcopy(nc.CrossingDF))

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon as MatplotPolygon

def GreatCircle(Start_p,End_p):
    import pyproj
    import numpy as np
    startlong, startlat = Start_p
    endlong, endlat     = End_p


    # calculate distance between points
    g = pyproj.Geod(ellps='WGS84')
    (az12, az21, dist) = g.inv(startlong, startlat, endlong, endlat)

    # calculate line string along path with segments <= 1 km
    lonlats = g.npts(startlong, startlat, endlong, endlat,
                    1 + int(dist / 1000))

    lonlats = np.array(lonlats)

    return lonlats



fig, ax = plt.subplots(1, 1, figsize=(25, 11))
for idx,row in nc.CrossingDF.iterrows():
    ax.add_patch(MatplotPolygon(TT.Mesh.cellBoxes[row.cellEnd.name].getBounds(), closed=True, fill=False, color='grey', alpha=1))
    ax.add_patch(MatplotPolygon(TT.Mesh.cellBoxes[row.cellStart.name].getBounds(), closed=True, fill=False, color='grey', alpha=1))

ax.plot(nc.CrossingDF['cX'],nc.CrossingDF['cY'],marker='o')

GC = GreatCircle(tuple(nc.CrossingDF.iloc[0][['cX','cY']]),tuple(nc.CrossingDF.iloc[-1][['cX','cY']]))
ax.plot(GC[:,0],GC[:,1],color='red')


In [ ]:
nc.CrossingDF

In [ ]:
nc._mergePoint()

In [ ]:
nc.CrossingDF

In [ ]:
#1. Find indices of points that are identical
Dist = np.sqrt((nc.CrossingDF['cX'][1:].to_numpy() - nc.CrossingDF['cX'][:-1].to_numpy())**2 + (nc.CrossingDF['cY'][1:].to_numpy() - nc.CrossingDF['cY'][:-1].to_numpy())**2)

idx = np.where(Dist<=)[0]
# id = 10 

# # if len(idx) != 0:
# #     #2. Remove the row and update thr case information and crossing point
# #     for id in idx:
# neighbourIndex=np.where(np.array(nc.CrossingDF.iloc[id]['cellStart']['neighbourIndex'])==nc.CrossingDF.iloc[id+1]['cellEnd'].name)[0][0]

# case = nc.CrossingDF['cellStart'].iloc[id]['case'][neighbourIndex]
# crossingPoint = nc.CrossingDF['cellStart'].iloc[id]['neighbourCrossingPoints'][neighbourIndex]


print(idx)

#         print('Merge Point Case={}'.format(case))

#         if abs(case)==1 or abs(case)==3:
#             self.CrossingDF['cX'].iloc[id]      = crossingPoint[0]
#             self.CrossingDF['cY'].iloc[id]      = crossingPoint[1]
#             self.CrossingDF['cellEnd'].iloc[id] = copy.deepcopy(self.CrossingDF.iloc[id+1]['cellEnd'])
#             self.CrossingDF['case'].iloc[id] = copy.deepcopy(case)
#             self.CrossingDF = self.CrossingDF.drop(self.CrossingDF.iloc[id+1].name).sort_index().reset_index(drop=True)
#             self.CrossingDF.index = np.arange(int(self.CrossingDF.index.min()),int(self.CrossingDF.index.max()*1e3 + 1e3),int(1e3))

In [ ]:
nc.CrossingDF

In [ ]:
id=10
nc.triplet = nc.CrossingDF.iloc[id:id+3]

fig, ax = plt.subplots(1, 1, figsize=(25, 11))
for idx,row in nc.triplet.iterrows():
    ax.add_patch(MatplotPolygon(TT.Mesh.cellBoxes[row.cellEnd.name].getBounds(), closed=True, fill=False, color='grey', alpha=1))
    ax.add_patch(MatplotPolygon(TT.Mesh.cellBoxes[row.cellStart.name].getBounds(), closed=True, fill=False, color='grey', alpha=1))

ax.plot(nc.triplet['cX'],nc.triplet['cY'],marker='o')
ax.scatter(-60.00,-60.05550092994375,15,'r',marker='o')

In [ ]:

def GreatCircle(Start_p,End_p):
    import pyproj
    import numpy as np
    startlong, startlat = Start_p
    endlong, endlat     = End_p


    # calculate distance between points
    g = pyproj.Geod(ellps='WGS84')
    (az12, az21, dist) = g.inv(startlong, startlat, endlong, endlat)

    # calculate line string along path with segments <= 1 km
    lonlats = g.npts(startlong, startlat, endlong, endlat,
                    1 + int(dist / 1000))

    lonlats = np.array(lonlats)

    return lonlats



map = Plot.BaseMap(logo=True,logoPos=[5,88])
map = Plot.MapMesh(cellGrid,map,threshold=OptInfo['Route']['MaxIceExtent'])
map = Plot.MapWaypoints(pd.read_csv(OptInfo['Route']['WayPoints']),map)


# PointsCheck = nc.CrossingDF[['cX','cY']].to_numpy()
# map = MapMPaths(PointsCheck,map,color='green',PathPoints=True)

for mPath in JavaPaths('pathDetails2013_IceEffectOff.json'):
    map = MapMPaths(mPath,map,color='red',PathPoints=False)

for Path in Paths:
    Points = Path[['cX','cY']].to_numpy()
    map = MapMPaths(Points,map,PathPoints=True)


# map = MapMPaths(GreatCircle((-69.24361111111114,-68.38916666666667),(-59.879999999999995,-52.63472222222222)),map,color='purple',PathPoints=False)
# map = MapMPaths(GreatCircle((-69.24361111111114,-68.38916666666667),(-129.0,-58.0)),map,color='purple',PathPoints=False)
# map = MapMPaths(GreatCircle((-69.24361111111114,-68.38916666666667),(-129.69083333333333,-69.32305555555556)),map,color='purple',PathPoints=False)



#map = Plot.MapPaths(Paths,map,PathPoints=False)
map = Plot.LayerControl(map,collapsed=True)
map